In [ ]:
import os
import sys

sys.path.insert(0, os.path.abspath("../.."))
from dotenv import load_dotenv

from examples.config import settings

load_dotenv(".env", override=True)
%load_ext autoreload
%autoreload 2

# Project Kickoff Agent

This notebook demonstrates the **planning → execution** separation that makes deepagents powerful.

## What Makes This Different

Unlike simple tool-calling agents, this agent:

1. **Plans First**: Uses `write_todos` to create a plan before doing any work
2. **Delegates**: Uses specialized subagents for different aspects of the project
3. **Creates Artifacts**: Saves deliverables to the virtual filesystem
4. **Synthesizes**: Combines all subagent outputs into a final project plan

## Architecture

```
                    ┌─────────────────────┐
                    │   ORCHESTRATOR    │
                    │   (Main Agent)    │
                    │                   │
                    │  1. Plan (todos)  │
                    │  2. Delegate      │
                    │  3. Synthesize    │
                    └────────┬────────────┘
                             │
           ┌─────────────┼─────────────┐
           │             │             │
           ▼             ▼             ▼
    ┌──────────┐  ┌──────────┐  ┌──────────┐
    │ ARCHITECT│  │  TASK    │  │   RISK   │
    │          │  │ PLANNER  │  │ ASSESSOR │
    └────┬─────┘  └────┬─────┘  └────┬─────┘
         │             │             │
         ▼             ▼             ▼
    architecture.md  tasks.md     risks.md
                    \\ | //
                     \\|//
                      ▼
               PROJECT_PLAN.md
```

## Deepagent Features Demonstrated

| Feature | How It's Used |
|---------|---------------|
| `write_todos` | Planning work before execution |
| `task` (subagents) | Delegating to architect, task-planner, risk-assessor |
| `write_file` | Saving deliverables to virtual filesystem |
| `read_file` | Reading subagent outputs for synthesis |
| Context isolation | Each subagent has focused context |

## The Orchestrator's Tools

The main agent has access to both **native deepagent tools** and **custom tools**.

In [ ]:
from examples.project_kickoff_agent.tools.tools import (
    get_deliverable_template,
    validate_project_plan,
)

print("Custom Tools:")
print(f"  - {get_deliverable_template.name}: Get markdown templates")
print(f"  - {validate_project_plan.name}: Check deliverables exist")
print()
print("Native Deepagent Tools (automatically available):")
print("  - write_todos: Plan work before executing")
print("  - write_file: Save deliverables to filesystem")
print("  - read_file: Read files from filesystem")
print("  - task: Delegate to subagents")

## The Subagents

Each subagent is a specialist with focused expertise and a specific deliverable.

In [ ]:
subagents = [
    ("architect", "Designs technical architecture", "/docs/architecture.md"),
    ("task-planner", "Creates work breakdown", "/docs/tasks.md"),
    ("risk-assessor", "Identifies project risks", "/docs/risks.md"),
]

print("Specialist Subagents:")
print("=" * 60)
for name, role, deliverable in subagents:
    print(f"\n{name}:")
    print(f"  Role: {role}")
    print(f"  Deliverable: {deliverable}")

## The Orchestrator's Instructions

The orchestrator is explicitly instructed to **plan first, then execute**.

In [ ]:
from examples.project_kickoff_agent.prompts.prompts import ORCHESTRATOR_INSTRUCTIONS
from examples.utils import show_prompt

# Show just the workflow section
workflow_section = ORCHESTRATOR_INSTRUCTIONS.split("## AVAILABLE TOOLS")[0]
show_prompt(workflow_section)

## Creating the Agent

The agent is created with custom tools and three specialized subagents.

In [ ]:
from deepagents import create_deep_agent
from langchain_google_genai import ChatGoogleGenerativeAI

from examples.project_kickoff_agent.prompts.prompts import (
    ORCHESTRATOR_INSTRUCTIONS,
    ARCHITECT_INSTRUCTIONS,
    TASK_PLANNER_INSTRUCTIONS,
    RISK_ASSESSOR_INSTRUCTIONS,
)

model = ChatGoogleGenerativeAI(model=settings.GOOGLE_MODEL_NAME, temperature=settings.TEMPERATURE)

# Define subagents
architect_subagent = {
    "name": "architect",
    "model": model,
    "description": "Delegate architecture design. Creates /docs/architecture.md",
    "system_prompt": ARCHITECT_INSTRUCTIONS,
    "tools": [get_deliverable_template],
}

task_planner_subagent = {
    "name": "task-planner",
    "model": model,
    "description": "Delegate task breakdown. Creates /docs/tasks.md",
    "system_prompt": TASK_PLANNER_INSTRUCTIONS,
    "tools": [get_deliverable_template],
}

risk_assessor_subagent = {
    "name": "risk-assessor",
    "model": model,
    "description": "Delegate risk assessment. Creates /docs/risks.md",
    "system_prompt": RISK_ASSESSOR_INSTRUCTIONS,
    "tools": [get_deliverable_template],
}

# Create the orchestrator agent
agent = create_deep_agent(
    model=model,
    tools=[get_deliverable_template, validate_project_plan],
    system_prompt=ORCHESTRATOR_INSTRUCTIONS,
    subagents=[architect_subagent, task_planner_subagent, risk_assessor_subagent],
)

print("Project Kickoff Agent created!")
print("  - Custom tools: 2")
print("  - Subagents: architect, task-planner, risk-assessor")
print("  - Native tools: write_todos, write_file, read_file, task, etc.")

## Visualize the Agent Graph

In [ ]:
from IPython.display import Image, display

display(Image(agent.get_graph().draw_mermaid_png()))

## Run the Agent: Full Project Kickoff

Watch how the agent:
1. **Plans** the kickoff process using `write_todos`
2. **Delegates** to each subagent using `task`
3. **Collects** deliverables from the filesystem
4. **Synthesizes** the final project plan

In [ ]:
from examples.utils import format_messages

print("=" * 80)
print("PROJECT KICKOFF: Customer Feedback Analytics Platform")
print("=" * 80)
print()

result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": """Create a complete project kickoff package for:

**Project**: Customer Feedback Analytics Platform

**Description**: A platform that collects customer feedback from multiple channels
(email, chat, surveys, social media), uses NLP to analyze sentiment and extract
topics, and provides dashboards for product and support teams.

**Key Requirements**:
- Real-time feedback ingestion
- Sentiment analysis with 90%+ accuracy
- Topic clustering and trend detection
- Role-based dashboards
- Integration with existing CRM (Salesforce)

Please create the full project plan with architecture, tasks, and risk assessment.""",
            }
        ],
    },
)

format_messages(result["messages"])

## Examine the Generated Deliverables

The agent created files in its virtual filesystem. Let's examine them.

In [ ]:
from deepagents.backends.utils import file_data_to_string

print("Generated Files:")
print("=" * 40)

if "files" in result:
    for path in sorted(result["files"].keys()):
        file_data = result["files"][path]
        content = file_data_to_string(file_data)
        lines = len(content.split("\n"))
        print(f"  {path} ({lines} lines)")
else:
    print("  No files generated (check agent output)")

In [ ]:
# Show the architecture document
if "files" in result and "/docs/architecture.md" in result["files"]:
    content = file_data_to_string(result["files"]["/docs/architecture.md"])
    show_prompt(content[:3000] + "\n\n... [truncated]" if len(content) > 3000 else content)

In [ ]:
# Show the task breakdown
if "files" in result and "/docs/tasks.md" in result["files"]:
    content = file_data_to_string(result["files"]["/docs/tasks.md"])
    show_prompt(content[:3000] + "\n\n... [truncated]" if len(content) > 3000 else content)

In [ ]:
# Show the risk assessment
if "files" in result and "/docs/risks.md" in result["files"]:
    content = file_data_to_string(result["files"]["/docs/risks.md"])
    show_prompt(content[:3000] + "\n\n... [truncated]" if len(content) > 3000 else content)

## Analyze the Agent's Behavior

Let's examine how the agent followed the planning → execution pattern.

In [ ]:
# Count tool usage
tool_usage = {}
subagent_calls = []

for msg in result["messages"]:
    if hasattr(msg, "tool_calls") and msg.tool_calls:
        for tc in msg.tool_calls:
            name = tc.get("name", "unknown")
            tool_usage[name] = tool_usage.get(name, 0) + 1
            if name == "task":
                subagent_type = tc.get("args", {}).get("subagent_type", "unknown")
                subagent_calls.append(subagent_type)

print("Agent Behavior Analysis")
print("=" * 40)
print()
print("Tool Usage:")
for tool, count in sorted(tool_usage.items()):
    marker = " <- Planning!" if tool == "write_todos" else ""
    marker = " <- Delegation!" if tool == "task" else marker
    marker = " <- Artifacts!" if tool in ["write_file", "read_file"] else marker
    print(f"  {tool}: {count}{marker}")

print()
print("Subagent Delegations:")
for sa in subagent_calls:
    print(f"  -> {sa}")

print()
print("Files Created:", len(result.get("files", {})))

## Summary

This Project Kickoff Agent demonstrates the **power of deepagents**:

### 1. Planning Before Execution
The agent uses `write_todos` to create a structured plan before doing any work.
This ensures systematic, predictable behavior.

### 2. Specialized Delegation
Instead of doing everything itself, the agent delegates to specialists:
- **architect**: Focuses purely on system design
- **task-planner**: Focuses purely on work breakdown
- **risk-assessor**: Focuses purely on risk analysis

Each subagent has **context isolation** - they only see what's relevant to their task.

### 3. Artifact Generation
The agent creates persistent deliverables in the virtual filesystem:
- `/docs/architecture.md`
- `/docs/tasks.md`
- `/docs/risks.md`
- `/docs/PROJECT_PLAN.md`

### 4. Synthesis
The orchestrator combines all subagent outputs into a cohesive final deliverable.

---

This pattern can be applied to many complex workflows:
- **Code Review**: Plan areas to review → Delegate to security/performance/style reviewers → Synthesize findings
- **Research**: Plan topics → Delegate research to specialists → Compile report
- **Migration**: Plan phases → Execute each phase → Validate and report